# Compiler Comparison

Performance comparison between GCC and Clang across all benchmarks.

This notebook presents tabular data comparing the two compilers.

In [ ]:
import sys
from pathlib import Path

import polars as pl

sys.path.insert(0, str(Path.cwd().parent.parent / "src"))
from benchmarks import BuildType, load_benchmark

In [ ]:
repo_path = Path("../../..")

# Load all benchmarks
benchmark_names = [
    ("allocs", "Allocators"),
    ("containers_queue", "Queue"),
    ("containers_vector", "Vector"),
    ("containers_set", "Set"),
    ("containers_map", "Map"),
]

all_data = []

for bench_name, bench_label in benchmark_names:
    df = load_benchmark(repo_path, bench_name)
    if not df.is_empty():
        df = df.filter(pl.col("build_type") == BuildType.RELEASE.value)
        df = df.with_columns(pl.lit(bench_label).alias("benchmark_category"))
        all_data.append(df)

# Combine all benchmark data
combined_df = pl.concat(all_data) if all_data else pl.DataFrame()

## Compiler Comparison Table

The table below shows performance metrics for each test, comparing GCC vs Clang.
The "Clang Speedup" column shows the relative performance (Clang / GCC), where >1.0 means Clang is faster.

In [ ]:
if not combined_df.is_empty():
    # Pivot data to compare compilers
    comparison_df = (
        combined_df.group_by(["benchmark_category", "test_name", "name", "size"])
        .agg(
            pl.col("items_per_second_M")
            .filter(pl.col("compiler") == "gcc")
            .mean()
            .alias("gcc_throughput"),
            pl.col("items_per_second_M")
            .filter(pl.col("compiler") == "clang")
            .mean()
            .alias("clang_throughput"),
        )
        .with_columns(
            (pl.col("clang_throughput") / pl.col("gcc_throughput")).alias(
                "clang_speedup"
            )
        )
        .sort(["benchmark_category", "test_name", "name", "size"])
    )

    # Format for display - show all benchmarks
    display_df = comparison_df.with_columns(
        pl.col("gcc_throughput").round(2).alias("GCC (M/s)"),
        pl.col("clang_throughput").round(2).alias("Clang (M/s)"),
        pl.col("clang_speedup").round(3).alias("Clang Speedup"),
    ).select(
        [
            pl.col("benchmark_category").alias("Category"),
            pl.col("test_name").alias("Test"),
            pl.col("name").alias("Implementation"),
            pl.col("size").alias("Size"),
            "GCC (M/s)",
            "Clang (M/s)",
            "Clang Speedup",
        ]
    )

    print(f"Total comparisons: {len(display_df)}")
    print(f"Showing all {len(display_df)} benchmark results\n")
    display_df
else:
    print("No benchmark data found")

## Summary Statistics

In [ ]:
if not combined_df.is_empty() and "clang_speedup" in comparison_df.columns:
    summary = (
        comparison_df.group_by("benchmark_category")
        .agg(
            [
                pl.col("clang_speedup").mean().alias("avg_speedup"),
                pl.col("clang_speedup").min().alias("min_speedup"),
                pl.col("clang_speedup").max().alias("max_speedup"),
                pl.len().alias("num_tests"),
            ]
        )
        .with_columns(
            pl.col("avg_speedup").round(3),
            pl.col("min_speedup").round(3),
            pl.col("max_speedup").round(3),
        )
        .select(
            [
                pl.col("benchmark_category").alias("Category"),
                pl.col("avg_speedup").alias("Avg Speedup"),
                pl.col("min_speedup").alias("Min Speedup"),
                pl.col("max_speedup").alias("Max Speedup"),
                pl.col("num_tests").alias("Tests"),
            ]
        )
        .sort("Category")
    )

    print("\nSummary by benchmark category:")
    print("Speedup > 1.0 means Clang is faster, < 1.0 means GCC is faster\n")
    summary